# Uniform interface for API Calls to different LLMs

Autogen provides an uniform interface for API calls to different LLMs. You can also create different agents with different LLMs.

## Config list setup
In the setup below, we first create an `OAI_CONFIG_LIST` for different LLMs. 

```python
[
    {   
        # using OpenAI API
        "model": "gpt-35-turbo-1106",
        "api_key": "YOUR_API_KEY",
    },
    {
        # using Azure API
        "model": "gpt-4-turbo-1106",
        "api_key": "YOUR_API_KEY",
        "api_type": "azure",
        "base_url": "YOUR_BASE_URL",
        "api_version": "YOUR_API_VERSION"
    },
    {   
        # using Google API
        "model": "gemini-1.5-pro-latest",
        "api_key": "YOUR_API_KEY",
        "api_type": "google",
    },
    {
        # using DeepInfra
        "model": "meta-llama/Meta-Llama-3-70B-Instruct",
        "api_key": "YOUR_API_KEY",
    },
    {
        # using DeepInfra
        "model": "mistralai/Mixtral-8x22B-Instruct-v0.1", 
        "api_key": "YOUR_API_KEY",
    }
]


## Uniform API to call different LLMs

In [2]:
import autogen
from autogen import OpenAIWrapper

def model_call_example_function(
        model: str,
        message: str,
        cache_seed: int = 41,
):
    """
    A helper function that demonstrates how to call different models using the OpenAIWrapper class.
    Note the name `OpenAIWrapper` is not accurate, as now it is a wrapper for multiple models, not just OpenAI. 
    This might be changed in the future.
    """
    config_list = autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={
            "model": [model],
        }
    )
    client = OpenAIWrapper(config_list=config_list)
    response = client.create(
        messages=[{"role": "user", "content": message}],
        cache_seed=cache_seed
    )

    print(f"Response from model {model}: {response.choices[0].message.content}")

    client.print_usage_summary()

In [18]:
model_call_example_function(
    model="gpt-35-turbo-1106",
    message="Tell me a joke."
)

Response from model gpt-35-turbo-1106: Why don't skeletons fight each other?

They don't have the guts.
----------------------------------------------------------------------------------------------------
No actual cost incurred (all completions are using cache).

Usage summary including cached usage: 
Total cost: 3e-05
* Model 'gpt-35-turbo': cost: 3e-05, prompt_tokens: 12, completion_tokens: 15, total_tokens: 27
----------------------------------------------------------------------------------------------------


In [19]:
model_call_example_function(
    model="gemini-1.5-pro-latest",
    message="Tell me a joke."
)

Response from model gemini-1.5-pro-latest: Why don't scientists trust atoms? 

Because they make up everything! 

----------------------------------------------------------------------------------------------------
No actual cost incurred (all completions are using cache).

Usage summary including cached usage: 
Total cost: 0.00042
* Model 'gemini-1.5-pro-latest': cost: 0.00042, prompt_tokens: 6, completion_tokens: 18, total_tokens: 24
----------------------------------------------------------------------------------------------------


In [3]:
model_call_example_function(
    model="meta-llama/Meta-Llama-3-70B-Instruct",
    message="Tell me a joke. "
)

Response from model meta-llama/Meta-Llama-3-70B-Instruct: Why couldn't the bicycle stand up by itself?

Because it was two-tired!

(Sorry, I know it's a bit of a "dad" joke, but I hope it made you smile!)
----------------------------------------------------------------------------------------------------
Usage summary excluding cached usage: 
Total cost: 4e-05
* Model 'meta-llama/Meta-Llama-3-70B-Instruct': cost: 4e-05, prompt_tokens: 15, completion_tokens: 41, total_tokens: 56

All completions are non-cached: the total cost with cached completions is the same as actual cost.
----------------------------------------------------------------------------------------------------


## Using different LLMs in agents

In [16]:
config_list_gpt_35 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-35-turbo-1106"],
    }
)

config_list_gemini = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-1.5-pro-latest"],
    }
)

config_list_llama3 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["meta-llama/Meta-Llama-3-70B-Instruct"],
    }
)

config_list_mistral = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["mistralai/Mixtral-8x22B-Instruct-v0.1"],
    }
)

In [17]:
gpt35_agent = autogen.AssistantAgent(
    name="Bob",
    system_message="You are in the Affirmative team of a debate. When it is your turn, please give at least one reason why you are for the topic.",
    llm_config={
        "cache_seed": 41,
        "config_list": config_list_gpt_35
    }
)

llama_agent = autogen.AssistantAgent(
    name="Alice",
    system_message="You are in the Negative team of a debate. The affirmative team has given their reason, please counter their argument.",
    llm_config={
        "cache_seed": 41,
        "config_list": config_list_llama3
    }
)

mistral_agent = autogen.AssistantAgent(
    name="Charlie",
    system_message="You are in the Affirmative team of a debate. When it is your turn, please give at least one reason why you are for the topic, or counter the negative team's argument.",
    llm_config={
        "cache_seed": 41,
        "config_list": config_list_mistral
    }
)

user_proxy = autogen.UserProxyAgent(
    name="User",
    code_execution_config=False,
)

groupchat = autogen.GroupChat(agents=[llama_agent, mistral_agent, gpt35_agent, user_proxy], messages=[], max_round=5, speaker_selection_method="round_robin")
manager = autogen.GroupChatManager(groupchat=groupchat)


In [19]:
chat_history = user_proxy.initiate_chat(
    recipient=manager,
    message="Debate Topic: Should vaccination be mandatory?"
)

User (to chat_manager):

Debate Topic: Should vaccination be mandatory?

--------------------------------------------------------------------------------

Next speaker: Alice

Alice (to chat_manager):

Thank you, honorable judges, worthy opponents, and fellow debaters. As the Negative team, I'm excited to counter the Affirmative team's argument that vaccination should be mandatory.

While I understand the intention behind making vaccination mandatory, I strongly believe that individual freedom and autonomy should not be compromised. Mandatory vaccination policies would infringe upon a person's right to make informed decisions about their own health and well-being.

Firstly, the Affirmative team may argue that mandatory vaccination is necessary to protect public health. However, it's essential to recognize that vaccines are not 100% effective, and outbreaks can still occur even with high vaccination rates. Moreover, there are alternative measures to achieve herd immunity, such as educat